In [1]:
# python
import sys
import os
import importlib
# columnar analysis
import awkward as ak
from dask.distributed import Client
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import sidm_processor, utilities, cutflow, scaleout
# always reload local modules to pick up changes during development
importlib.reload(sidm_processor)
importlib.reload(utilities)
importlib.reload(scaleout)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()
%matplotlib inline

In [2]:
samples = [   
    "4mu_800GeV_0p25GeV_2p5mm",
]

fileset = utilities.make_fileset(samples, "llpNanoAOD_v2", max_files=1)

In [3]:
runner = processor.Runner(
    executor=processor.IterativeExecutor(),
    #executor=processor.DaskExecutor(),
    #executor=processor.FuturesExecutor(),
    schema=NanoAODSchema,
    # maxchunks=1,
)

hist_menu = utilities.load_yaml("../configs/hist_collections.yaml")
hist_choice = "test"
collection = utilities.flatten(hist_menu[hist_choice]) #To change the histograms used, swap "base" for the other collections
p = sidm_processor.SidmProcessor(
    [
        "matchedLj",
    ],
    [hist_choice],
    lj_reco_choices=["0.4"],
    llpnanoaod=True,
)
output = runner.run(fileset, treename='Events', processor_instance=p)
out = output["out"]

Output()

Output()

Exception: Failed processing file: WorkItem(dataset='4mu_800GeV_0p25GeV_2p5mm', filename='root://xcache//store/group/lpcmetx/SIDM/ULSignalSamples/2018_v6/LLPNanoAOD_V2/SIDM_BsTo2DpTo4Mu_MBs-800_MDp-0p25_ctau-2p5/LLPnanoAODv2/SIDM_BsTo2DpTo4Mu_MBs-800_MDp-0p25_ctau-2p5_part-0.root', treename='Events', entrystart=0, entrystop=842, fileuuid=b'R\x03d\xf6#w\x11\xef\x95\x9d,\x15\xe6\x9b\xbe\xef', usermeta={})

In [ ]:
cutflow.print_multi_table([out[sample]["cutflow"]["4mutest"] for sample in samples], [sample for sample in samples])

In [ ]:
plt.subplots(1,len(samples),figsize=(len(samples)*6,6))
for i in range(len(samples)):
    plt.subplot(1,len(samples),i+1)
    utilities.plot(out[samples[i]]["hists"]["genE_pt"]["baseNoLj", :20j], density=True, flow='none')